In [ ]:
from deephive.optimizers import deephive as dh 
from deephive.environment.utils import * 
from datetime import datetime
import os 
import neptune 
from deephive.environment.deephive_utils import *
import torch

In [ ]:
config_path = "../config/exp_config.json"
model_path = "../models/pbest_unfreeze.pth"
model_path_2 = "../models/gbest.pth"

def initialize_logger(api_token, tags, config, mode="train"):
    run = neptune.init_run(
    project="DMO-LAB/DeepHive-V2",
    source_files=["environment", "policies", "deephive.py", "config"],
    api_token=api_token,
    tags=[tags, mode, config["objective_function"], str(config["layer_size"])]
    )
    neptune_logger = run
    return neptune_logger

def get_action(observation_info, agent_policy, env, observation_std=None, random=False):
    # Ensure observation_info is a numpy array
    
    if random:
        print("Random action")
        return env.action_space.sample()
    
    observation = observation_info
    if not isinstance(observation, np.ndarray):
        observation = np.array(observation)
        assert observation.shape[0] == env.n_dim, "Observation must have the same number of dimensions as the environment"

    # Initialize observation_std with zeros or use provided std, ensuring it matches the shape of observation
    if observation_std is None:
        observation_std = np.zeros_like(observation)
    else:
        observation_std = np.array(observation_std)

    # Convert observations and stds to the appropriate format (Flatten if necessary)
    # Assuming observation and observation_std are 2D arrays of shape (n_agents, n_dim)
    # Flatten observation and observation_std for processing
    observation_flat = observation.reshape(env.n_dim * env.n_agents, -1)  # Flatten to 1D array
    observation_std_flat = observation_std.reshape(-1)  # Flatten to 1D array
    # Pass the entire flattened observation and std arrays to select_action
    action_flat = agent_policy.select_action(observation_flat, observation_std_flat)

    # Reshape the flattened action array back to the original (n_agents, n_dim) shape
    actions = action_flat.reshape(env.n_dim, env.n_agents).T  # Reshape to (n_agents, n_dim

    return actions  # Return the action



In [ ]:
config = parse_config(config_path)
config["update_timestep"] = 25 
config["decay_rate"] = 0.99
config["n_dim"] = 2
config["include_gbest"] = False 
if config["include_gbest"]:
    config["obs_dim"] = 11
config["n_agents"] = 10
config["ep_length"] = 25
config["test_ep_length"] = 100
config["log_interval"] = 500
config["decay_interval"] = 1000
config["save_interval"] = 25000
config["min_action_std"] = 0.02
config["n_episodes"] = 5000
config["update_start"] = 0
config["action_std"] = 0.3
config["variable_std"]  = False
config["reward_scheme"] = "FullRewardScheme"
config['use_gbest'] = True
config['objective_function'] = "SphereFunction"
config["function_id"] = "f01"
config["use_optimal_value"] = True 
config["log_scale"] = False
mode = "test"
log = False 
tags = "new reward training"
env, agent_policy = initialize(config, mode=mode, model_path=model_path_2)
neptune_logger = None

In [ ]:
obs = env.reset()
obs = obs[0]

In [ ]:
agent_policy.set_action_std(0.00001)

In [ ]:
actions = get_action(obs, agent_policy, env, random=False)
obs, reward, done, info = env.step(actions)
obs = obs[0]
env.render()

In [ ]:
env.pbest

## DEEPHIVE TRAINER

In [ ]:
import argparse

In [ ]:
config_path = "../config/exp_config.json"
model_path = "../models/pbest_unfreeze.pth"
model_path_2 = "../models/gbest.pth"

config = parse_config(config_path)

config["update_timestep"] = 1000
config["decay_rate"] = 0.95
config["log_interval"] = 500
config["decay_interval"] = 2000
config["save_interval"] = 2000
config["min_action_std"] = 0.02
config["n_episodes"] = 5000
config["update_start"] = 0
config["action_std"] = 0.2
config["variable_std"]  = False
config["reward_scheme"] = "FullRewardScheme"
config["use_gbest"] = False

deephive_trainer = dh.OptimizationTrainer(config, mode="train")

In [ ]:
deephive_trainer.train_agent(title="old_reward_pbest")